# CS6700 : Programming Assignment - 3
Students:
- Janmenjaya Panda (ME20B087)
- Nishant Sahoo (ME20B122)

In [ ]:
# Importing Libraries
import os
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
import random
import time
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.makedirs('plots', exist_ok=True)
os.makedirs('results', exist_ok=True)

In [ ]:
gamma = 0.9 # Discount Factor

# Defining the Environment
env = gym.make("Taxi-v3", render_mode="rgb_array")
# render modes : human, rgb_array, ansi
env.reset()
# env.render()

print("Current State : ", env.s)

# Print number of states and actions
print("Number of States : ", env.observation_space.n)
print("Number of Actions : ", env.action_space.n)

actions = {0 : "South", 1 : "North", 2 : "East", 3 : "West", 4 : "Pickup", 5 : "Dropoff"}

next_state, reward, done, info = env.step(1)
print("Action : ", actions[1])
print("Next State : ", next_state)
print("Reward : ", reward)
print("Done : ", done)
print("Info : ", info)

In [ ]:
def decode_state(state):
    row, col, _, _ = env.decode(state)
    return [row, col]

dest_loc = {"R": [0,0], "G": [0,4], "Y": [4,0], "B": [4,3]}
loc_idx = {"R": 0, "G": 1, "Y": 2, "B": 3, "in_taxi": 4}


In [ ]:
def egreedy_policy(Q, state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return env.action_space.sample()
    else:
        return np.argmax(Q[state, :])


In [ ]:
class Option:
    '''
    An option is tuple of <I, pi, beta> where
    I: initiation set
    pi: policy
    beta: termination set
    '''
    def __init__(self, initiation_set, actions, termination_set, name):
        self.initiation_set = initiation_set  # States where the option can be initiated
        self.actions = actions  # Actions that the option can take
        self.termination_set = termination_set  # States where the option terminates
        self.name = name  # Name of the option for identification
        self.q_values = np.zeros((500, 4)) # Q-values for each state and primitive action
        self.optdone = False

    def act(self, state, epsilon = 0.1): # not needed actually (can use epsilon greedy policy function instead)
        if np.random.rand() < epsilon:
            return np.random.choice(self.actions)
        else:
            return np.argmax(self.q_values[state])
        
    def done(self, state):
        optdone = state in self.termination_set
        return optdone
    

# Define the initiation and termination sets for each option
termination_sets = [set() for _ in range(4)] # R, G, Y, B

for state in range(500):
    if decode_state(state) == dest_loc["R"]: # If the taxi is at the red location
        termination_sets[0].add(state) 
    if decode_state(state) == dest_loc["G"]:
        termination_sets[1].add(state)
    if decode_state(state) == dest_loc["Y"]:
        termination_sets[2].add(state)
    if decode_state(state) == dest_loc["B"]:
        termination_sets[3].add(state)

initiation_sets = [set(range(500)) - term_set for term_set in termination_sets]

# Define the options
options = [] # List of options: Move_to_R, Move_to_G, Move_to_Y, Move_to_B

for loc, idx in loc_idx.items():
    if loc != "in_taxi":  # Skip the "in_taxi" location
        initiation_set = initiation_sets[idx]
        termination_set = termination_sets[idx]
        option_name = f"Move_to_{loc}"
        option = Option(initiation_set, list(range(4)), termination_set, option_name)
        options.append(option)


In [ ]:
print(options[0].termination_set)
print(options[1].termination_set)
print(options[2].termination_set)
print(options[3].termination_set)


In [ ]:
# Implementing SMDP Q-Learning

def smdp_q_learning(env, options, num_episodes=1000, alpha=0.1, epsilon=0.001):
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    rewards = []
    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        total_reward = 0
        done = False
        while not done:
            eps = max(0.98**episode, 1e-6)
            action = egreedy_policy(Q, state, eps)
            
            # if primitive action
            if action < env.action_space.n:
                # Perform regular Q-Learning update
                next_state, reward, done, _ = env.step(action)
                Q[state][action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action])
                state = next_state
                total_reward += reward
            else: # if option
                option = options[action - env.action_space.n]
                initial_state = state
                option_reward = 0
                discount = 1
                steps = 0
                while not option.done(state) and steps < 100:
                    optact = option.act(state)
                    next_state, reward, done, _ = env.step(optact)
                    option_reward += discount * reward
                    discount *= gamma
                    steps += 1
                    temp_reward = option_reward
                    if option.done(next_state):
                        temp_reward += 20 # Reward for completing the option
                    # Update the Q-value within the option
                    option.q_values[state][optact] += alpha * (temp_reward + gamma * np.max(option.q_values[next_state]) - option.q_values[state][optact])
                    state = next_state

                # Update the Q-value of the option
                Q[initial_state][action] += alpha * (option_reward + discount * np.max(Q[state]) - Q[initial_state][action])
                total_reward += option_reward
        rewards.append(total_reward)
    return Q, rewards

# Training the SMDP Q-Learning Agent
Q, rewards = smdp_q_learning(env, options, num_episodes=1500, alpha=0.1, epsilon=0.001)
# Save the Q-values and rewards
np.save("results/smdp_q_learning.npy", Q)
np.save("results/smdp_q_learning_rewards.npy", rewards)

# Plotting the Rewards
plt.plot(rewards)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("SMDP Q-Learning Rewards")
plt.show()

# # Visualizing the Q-Values
# plt.figure(figsize=(15, 10))
# for i in range(6):
#     plt.subplot(2, 3, i + 1)
#     sns.heatmap(Q[:, i].reshape(25, 20), cmap="coolwarm", cbar=False)
#     plt.title(f"Action {actions[i]}")
# plt.suptitle("SMDP Q-Learning Q-Values")
# plt.tight_layout()
# plt.subplots_adjust(top=0.9)
# plt.show()


In [ ]:
# Implementing Intra-Option Q-Learning
# Logic:
# At every step, the state-action value for the 
# primitive action as well as the state-action value 
# for all options that would have selected the same 
# action are updated, regardless of the option in 
# effect.

# Let pie_0 be : a1 (for s1), a2 (for s2), a3 (for s3), ...
#  for options:
# Q(s1, o) = Q(s1, o) + alpha * (r1 + gamma * Q(s2, o) - Q(s1, o)) -> if o does not terminate in s2
# Q(s1, o) = Q(s1, o) + alpha * (r1 + gamma * max_a (Q(s2, a)) - Q(s1, o)) -> if o terminates in s2

def intra_option_q_learning(env, options, num_episodes=1000, alpha=0.1, epsilon=0.1):
    Q = np.zeros((env.observation_space.n, env.action_space.n + len(options)))
    rewards = []
    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        total_reward = 0
        done = False
        while not done:
            eps = max(0.98**episode, 1e-6)
            action = egreedy_policy(Q, state, eps)
            
            # if primitive action
            if action < env.action_space.n:
                # Perform regular Q-Learning update
                next_state, reward, done, _ = env.step(action)
                Q[state][action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][action])
                total_reward += reward
            
            # Update state-action values of options that would have selected the same action
            for option in options:
                if action in option.actions:
                    next_state, reward, done, _ = env.step(option.act(state))
                    if option.done(next_state): # if option terminates in next state
                        Q[state][env.action_space.n + options.index(option)] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state][env.action_space.n + options.index(option)])
                    else: # if option does not terminate in next state
                        Q[state][env.action_space.n + options.index(option)] += alpha * (reward + gamma * Q[next_state][env.action_space.n + options.index(option)] - Q[state][env.action_space.n + options.index(option)])
                        
                    # Update the Q-value within the option
                    option.q_values[state][action] += alpha * (reward + gamma * np.max(option.q_values[next_state]) - option.q_values[state][action])
                    
            state = next_state
        rewards.append(total_reward)
    return Q, rewards

# Training the Intra-Option Q-Learning Agent
Q, rewards = intra_option_q_learning(env, options, num_episodes=1500, alpha=0.1, epsilon=0.1)
# Save the Q-values and rewards
np.save("results/intra_option_q_learning.npy", Q)
np.save("results/intra_option_q_learning_rewards.npy", rewards)

# Plotting the Rewards
plt.plot(rewards)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("Intra-Option Q-Learning Rewards")
plt.show()
